In [1]:
import pandas as pd
import requests
import bs4
import re
from neo4j import GraphDatabase

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
src1 = requests.get('https://u.gg/lol/champions')
soup1 = bs4.BeautifulSoup(src1.text , 'lxml')
li = soup1.find('div', {'class':'champions-container'})
a_tags = li.find_all('a')
link_rows = []
name_rows = []
url_prefix = 'https://u.gg'
for i in a_tags:
    link = url_prefix + i['href']
    link_rows.append(link)
    name = i.find('img')['alt']
    name_rows.append(name)


In [6]:
primary_rune_tree_list = []
primary_rune_keystone_list = []
primary_rune_rune_list = []
secondary_rune_tree_list = []
secondary_rune_rune_list = []
shard_rune_rune_list = []
suggest_spell_text_list = []

for i in link_rows:
    src2 = requests.get(i)
    soup2 = bs4.BeautifulSoup(src2.text, 'lxml')
    rune_detail = soup2.find('div', {'class':'content-section_content recommended-build_runes'})

    #primary rune
    primary_rune_rune = []
    
    primary_rune = rune_detail.find('div', {'class':'rune-tree_v2 primary-tree'})
    primary_rune_tree = primary_rune.find('img')['alt'].replace('The Rune Tree ', '')
    primary_rune_tree_list.append(primary_rune_tree)
    primary_rune_k = primary_rune.find('div', {'class':'perk keystone perk-active'})
    primary_rune_keystone = primary_rune_k.find('img')['alt'].replace('The Keystone ', '')
    primary_rune_keystone_list.append(primary_rune_keystone)
    primary_rune_r = primary_rune.find_all('div', {'class':'perk perk-active'})
    for i in primary_rune_r:
        e = i.find('img')['alt']
        primary_rune_rune.append(e.replace('The Rune ', ''))
    primary_rune_rune_list.append(primary_rune_rune)

    #secondary rune
    secondary_rune_rune = []

    secondary_rune = rune_detail.find('div', {'class':'secondary-tree'})
    secondary_rune_t = secondary_rune.find('div', {'class':'rune-image-container'})
    secondary_rune_tree = secondary_rune_t.find('img')['alt'].replace('The Rune Tree ', '')
    secondary_rune_tree_list.append(secondary_rune_tree)
    secondary_rune_r = secondary_rune.find_all('div', {'class':'perk perk-active'})
    for i in secondary_rune_r:
        e = i.find('img')['alt']
        secondary_rune_rune.append(e.replace('The Rune ', ''))
    secondary_rune_rune_list.append(secondary_rune_rune)

    #shard rune
    shard_rune_rune = []

    shard_rune = rune_detail.find('div', {'class':'rune-tree_v2 stat-shards-container_v2'})
    shard_rune_r = shard_rune.find_all('div', {'class':'shard shard-active'})
    for i in shard_rune_r:
        e = i.find('img')['alt']
        shard_rune_rune.append(e.replace('The ', ''))
    shard_rune_rune_list.append(shard_rune_rune)

    #spell
    suggest_spell_text = []

    summoner_spell = soup2.find('div', {'class':'content-section_content summoner-spells'})
    suggest_spell = summoner_spell.find_all('img')
    for i in suggest_spell:
        e = i['alt']
        suggest_spell_text.append(e.replace('Summoner Spell ', ''))
    suggest_spell_text_list.append(suggest_spell_text)

#table = pd.DataFrame([primary_rune_tree_list,primary_rune_keystone_list,primary_rune_rune_list,secondary_rune_tree_list,secondary_rune_rune_list,shard_rune_rune_list,suggest_spell_text_list]).transpose()

In [7]:
primary_rune_rune_list_1 = []
primary_rune_rune_list_2 = []
primary_rune_rune_list_3 = []
secondary_rune_rune_list_1 = []
secondary_rune_rune_list_2 = []
shard_rune_rune_list_1 = []
shard_rune_rune_list_2 = []
shard_rune_rune_list_3 = []
suggest_spell_text_list_1 = []
suggest_spell_text_list_2 = []
for i in primary_rune_rune_list:
    primary_rune_rune_list_1.append(i[0])
    primary_rune_rune_list_2.append(i[1])
    primary_rune_rune_list_3.append(i[2])
for i in secondary_rune_rune_list:
    secondary_rune_rune_list_1.append(i[0])
    secondary_rune_rune_list_2.append(i[1])
for i in shard_rune_rune_list:
    shard_rune_rune_list_1.append(i[0])
    shard_rune_rune_list_2.append(i[1])
    shard_rune_rune_list_3.append(i[2])
for i in suggest_spell_text_list:
    suggest_spell_text_list_1.append(i[0])
    suggest_spell_text_list_2.append(i[1])





In [8]:
TableForChamp = pd.DataFrame([name_rows]).transpose()
TableForRecommendRune = pd.DataFrame([name_rows,primary_rune_tree_list,primary_rune_keystone_list,primary_rune_rune_list_1,primary_rune_rune_list_2,primary_rune_rune_list_3,secondary_rune_tree_list,secondary_rune_rune_list_1,secondary_rune_rune_list_2,shard_rune_rune_list_1,shard_rune_rune_list_2,shard_rune_rune_list_3,suggest_spell_text_list_1,suggest_spell_text_list_2]).transpose()


TableForRuneTreePrecision = pd.DataFrame(['Precision','Press the Attack','Lethal Tempo','Fleet Footwork','Conqueror','Overheal','Triumph','Presence of Mind','Legend: Alacrity','Legend: Tenacity','Legend: Bloodline','Coup de Grace','Cut Down','Last Stand']).transpose()
TableForRuneTreeDomination = pd.DataFrame(['Domination','Electrocute','Predator','Dark Harvest','Hail of Blades','Cheap Shot','Taste of Blood','Sudden Impact','Zombie Ward','Ghost Poro','Eyeball Collection','Treasure Hunter','Ingenious Hunter','Relentless Hunter','Ultimate Hunter']).transpose()
TableForRuneTreeSorcery = pd.DataFrame(['Sorcery','Summon Aery','Arcane Comet','Phase Rush','Nullifying Orb','Manaflow Band','Nimbus Cloak','Transcendence','Celerity','Absolute Focus','Scorch','Waterwalking','Gathering Storm']).transpose()
TableForRuneTreeResolve = pd.DataFrame(['Resolve','Grasp of the Undying','Aftershock','Guardian','Demolish','Font of Life','Shield Bash','Conditioning','Second Wind','Bone Plating','Overgrowth','Revitalize','Unflinching']).transpose()
TableForRuneTreeInspiration = pd.DataFrame(['Inspiration','Glacial Augment','Unsealed Spellbook','First Strike','Hextech Flashtraption','Magical Footwear','Perfect Timing','Future\'s Market','Minion Dematerializer','Biscuit Delivery','Cosmic Insight','Approach Velocity','Time Warp Tonic']).transpose()
TableForShardRune = pd.DataFrame(['Adaptive Force Shard','Attack Speed Shard','Scaling CDR Shard','Adaptive Force Shard','Armor Shard','Magic Resist Shard','Scaling Bonus Health Shard','Armor Shard','Magic Resist Shard'])
TableForSpell = pd.DataFrame(['Heal','Ghost','Barrier','Exhaust','Flash','Teleport','Smite','Cleanse','Ignite'])

In [100]:
#to create node
from requests.sessions import session
neo_data_champ = TableForChamp.values.tolist()
neo_data_recomment_rune = TableForRecommendRune.values.tolist()
neo_data_precision = TableForRuneTreePrecision.values.tolist()
neo_data_domination = TableForRuneTreeDomination.values.tolist()
neo_data_sorcery = TableForRuneTreeSorcery.values.tolist()
neo_data_resolve = TableForRuneTreeResolve.values.tolist()
neo_data_inspiration = TableForRuneTreeInspiration.values.tolist()
neo_data_spell = TableForSpell.values.tolist()
neo_exec_comm = []

for i in neo_data_champ:
  neo4j_create_statement_champ = "create (c:Champion {ChampionName :"+ repr(i[0]) +"})"
  neo_exec_comm.append(neo4j_create_statement_champ)
for i in neo_data_recomment_rune:
  neo4j_create_statement_rune_recommend = "create (Rrune:RecommentRune {ForChamp :"+ repr(i[0]) +", PrimaryRuneTree :"+ repr(i[1]) +", RuneKey :"+ repr(i[2]) +", PrimaryRune1 :"+ repr(i[3]) +", PrimaryRune2 :"+ repr(i[4]) +", PrimaryRune3 :"+ repr(i[5]) +", SecondaryRuneTree :"+ repr(i[6]) +", SecondaryRune1 :"+ repr(i[7]) +", SecondaryRune2 :"+ repr(i[8]) +", ShardRune1 :"+ repr(i[9]) +", ShardRune2 :"+ repr(i[10]) +", ShardRune3 :"+ repr(i[11]) +", Spell1 :"+ repr(i[12]) +", Spell2 :"+ repr(i[13]) +"})"
  neo_exec_comm.append(neo4j_create_statement_rune_recommend)
for i in neo_data_spell:
  neo4j_create_statement_spell = "create (spl:Spell {SpellName :"+ repr(i[0]) +"})"
  neo_exec_comm.append(neo4j_create_statement_spell)
for i in neo_data_precision:
  neo4j_create_statement_tree1 = "create (tree:RuneTree {RuneTreeName :"+ repr(i[0]) +", RuneKeystone1 :"+ repr(i[1]) +", RuneKeystone2 :"+ repr(i[2]) +", RuneKeystone3 :"+ repr(i[3]) +", RuneKeystone4 :"+ repr(i[4]) +", SubRune1 :"+ repr(i[5]) +", SubRune2 :"+ repr(i[6]) +", SubRune3 :"+ repr(i[7]) +", SubRune4 :"+ repr(i[8]) +", SubRune5 :"+ repr(i[9]) +", SubRune6 :"+ repr(i[10]) +", SubRune7 :"+ repr(i[11]) +", SubRune8 :"+ repr(i[12]) +", SubRune9 :"+ repr(i[13]) +"})"
  neo_exec_comm.append(neo4j_create_statement_tree1)
  neo4j_create_statement_key1 = "create (key:KeyStone {RuneKeystone :"+ repr(i[1]) +"})"
  neo_exec_comm.append(neo4j_create_statement_key1)
  neo4j_create_statement_key2 = "create (key:KeyStone {RuneKeystone :"+ repr(i[2]) +"})"
  neo_exec_comm.append(neo4j_create_statement_key2)
  neo4j_create_statement_key3 = "create (key:KeyStone {RuneKeystone :"+ repr(i[3]) +"})"
  neo_exec_comm.append(neo4j_create_statement_key3)
  neo4j_create_statement_key4 = "create (key:KeyStone {RuneKeystone :"+ repr(i[4]) +"})"
  neo_exec_comm.append(neo4j_create_statement_key4)
  neo4j_create_statement_sub1 = "create (sub:SubRune {SubRuneName :"+ repr(i[5]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub1)
  neo4j_create_statement_sub2 = "create (sub:SubRune {SubRuneName :"+ repr(i[6]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub2)
  neo4j_create_statement_sub3 = "create (sub:SubRune {SubRuneName :"+ repr(i[7]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub3)
  neo4j_create_statement_sub4 = "create (sub:SubRune {SubRuneName :"+ repr(i[8]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub4)
  neo4j_create_statement_sub5 = "create (sub:SubRune {SubRuneName :"+ repr(i[9]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub5)
  neo4j_create_statement_sub6 = "create (sub:SubRune {SubRuneName :"+ repr(i[10]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub6)
  neo4j_create_statement_sub7 = "create (sub:SubRune {SubRuneName :"+ repr(i[11]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub7)
  neo4j_create_statement_sub8 = "create (sub:SubRune {SubRuneName :"+ repr(i[12]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub8)
  neo4j_create_statement_sub9 = "create (sub:SubRune {SubRuneName :"+ repr(i[13]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub9)
for i in neo_data_domination:
  neo4j_create_statement_tree1 = "create (tree:RuneTree {RuneTreeName :"+ repr(i[0]) +", RuneKeystone1 :"+ repr(i[1]) +", RuneKeystone2 :"+ repr(i[2]) +", RuneKeystone3 :"+ repr(i[3]) +", RuneKeystone4 :"+ repr(i[4]) +", SubRune1 :"+ repr(i[5]) +", SubRune2 :"+ repr(i[6]) +", SubRune3 :"+ repr(i[7]) +", SubRune4 :"+ repr(i[8]) +", SubRune5 :"+ repr(i[9]) +", SubRune6 :"+ repr(i[10]) +", SubRune7 :"+ repr(i[11]) +", SubRune8 :"+ repr(i[12]) +", SubRune9 :"+ repr(i[13]) +", SubRune10 :"+ repr(i[14]) +"})"
  neo_exec_comm.append(neo4j_create_statement_tree1)
  neo4j_create_statement_key1 = "create (key:KeyStone {RuneKeystone :"+ repr(i[1]) +"})"
  neo_exec_comm.append(neo4j_create_statement_key1)
  neo4j_create_statement_key2 = "create (key:KeyStone {RuneKeystone :"+ repr(i[2]) +"})"
  neo_exec_comm.append(neo4j_create_statement_key2)
  neo4j_create_statement_key3 = "create (key:KeyStone {RuneKeystone :"+ repr(i[3]) +"})"
  neo_exec_comm.append(neo4j_create_statement_key3)
  neo4j_create_statement_key4 = "create (key:KeyStone {RuneKeystone :"+ repr(i[4]) +"})"
  neo_exec_comm.append(neo4j_create_statement_key4)
  neo4j_create_statement_sub1 = "create (sub:SubRune {SubRuneName :"+ repr(i[5]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub1)
  neo4j_create_statement_sub2 = "create (sub:SubRune {SubRuneName :"+ repr(i[6]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub2)
  neo4j_create_statement_sub3 = "create (sub:SubRune {SubRuneName :"+ repr(i[7]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub3)
  neo4j_create_statement_sub4 = "create (sub:SubRune {SubRuneName :"+ repr(i[8]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub4)
  neo4j_create_statement_sub5 = "create (sub:SubRune {SubRuneName :"+ repr(i[9]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub5)
  neo4j_create_statement_sub6 = "create (sub:SubRune {SubRuneName :"+ repr(i[10]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub6)
  neo4j_create_statement_sub7 = "create (sub:SubRune {SubRuneName :"+ repr(i[11]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub7)
  neo4j_create_statement_sub8 = "create (sub:SubRune {SubRuneName :"+ repr(i[12]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub8)
  neo4j_create_statement_sub9 = "create (sub:SubRune {SubRuneName :"+ repr(i[13]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub9)
  neo4j_create_statement_sub10 = "create (sub:SubRune {SubRuneName :"+ repr(i[14]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub10)
for i in neo_data_sorcery:
  neo4j_create_statement_tree1 = "create (tree:RuneTree {RuneTreeName :"+ repr(i[0]) +", RuneKeystone1 :"+ repr(i[1]) +", RuneKeystone2 :"+ repr(i[2]) +", RuneKeystone3 :"+ repr(i[3]) +", SubRune1 :"+ repr(i[4]) +", SubRune2 :"+ repr(i[5]) +", SubRune3 :"+ repr(i[6]) +", SubRune4 :"+ repr(i[7]) +", SubRune5 :"+ repr(i[8]) +", SubRune6 :"+ repr(i[9]) +", SubRune7 :"+ repr(i[10]) +", SubRune8 :"+ repr(i[11]) +", SubRune9 :"+ repr(i[12]) +"})"
  neo_exec_comm.append(neo4j_create_statement_tree1)
  neo4j_create_statement_key1 = "create (key:KeyStone {RuneKeystone :"+ repr(i[1]) +"})"
  neo_exec_comm.append(neo4j_create_statement_key1)
  neo4j_create_statement_key2 = "create (key:KeyStone {RuneKeystone :"+ repr(i[2]) +"})"
  neo_exec_comm.append(neo4j_create_statement_key2)
  neo4j_create_statement_key3 = "create (key:KeyStone {RuneKeystone :"+ repr(i[3]) +"})"
  neo_exec_comm.append(neo4j_create_statement_key3)
  neo4j_create_statement_sub1 = "create (sub:SubRune {SubRuneName :"+ repr(i[4]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub1)
  neo4j_create_statement_sub2 = "create (sub:SubRune {SubRuneName :"+ repr(i[5]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub2)
  neo4j_create_statement_sub3 = "create (sub:SubRune {SubRuneName :"+ repr(i[6]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub3)
  neo4j_create_statement_sub4 = "create (sub:SubRune {SubRuneName :"+ repr(i[7]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub4)
  neo4j_create_statement_sub5 = "create (sub:SubRune {SubRuneName :"+ repr(i[8]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub5)
  neo4j_create_statement_sub6 = "create (sub:SubRune {SubRuneName :"+ repr(i[9]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub6)
  neo4j_create_statement_sub7 = "create (sub:SubRune {SubRuneName :"+ repr(i[10]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub7)
  neo4j_create_statement_sub8 = "create (sub:SubRune {SubRuneName :"+ repr(i[11]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub8)
  neo4j_create_statement_sub9 = "create (sub:SubRune {SubRuneName :"+ repr(i[12]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub9)
for i in neo_data_resolve:
  neo4j_create_statement_tree1 = "create (tree:RuneTree {RuneTreeName :"+ repr(i[0]) +", RuneKeystone1 :"+ repr(i[1]) +", RuneKeystone2 :"+ repr(i[2]) +", RuneKeystone3 :"+ repr(i[3]) +", SubRune1 :"+ repr(i[4]) +", SubRune2 :"+ repr(i[5]) +", SubRune3 :"+ repr(i[6]) +", SubRune4 :"+ repr(i[7]) +", SubRune5 :"+ repr(i[8]) +", SubRune6 :"+ repr(i[9]) +", SubRune7 :"+ repr(i[10]) +", SubRune8 :"+ repr(i[11]) +", SubRune9 :"+ repr(i[12]) +"})"
  neo_exec_comm.append(neo4j_create_statement_tree1)
  neo4j_create_statement_key1 = "create (key:KeyStone {RuneKeystone :"+ repr(i[1]) +"})"
  neo_exec_comm.append(neo4j_create_statement_key1)
  neo4j_create_statement_key2 = "create (key:KeyStone {RuneKeystone :"+ repr(i[2]) +"})"
  neo_exec_comm.append(neo4j_create_statement_key2)
  neo4j_create_statement_key3 = "create (key:KeyStone {RuneKeystone :"+ repr(i[3]) +"})"
  neo_exec_comm.append(neo4j_create_statement_key3)
  neo4j_create_statement_sub1 = "create (sub:SubRune {SubRuneName :"+ repr(i[4]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub1)
  neo4j_create_statement_sub2 = "create (sub:SubRune {SubRuneName :"+ repr(i[5]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub2)
  neo4j_create_statement_sub3 = "create (sub:SubRune {SubRuneName :"+ repr(i[6]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub3)
  neo4j_create_statement_sub4 = "create (sub:SubRune {SubRuneName :"+ repr(i[7]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub4)
  neo4j_create_statement_sub5 = "create (sub:SubRune {SubRuneName :"+ repr(i[8]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub5)
  neo4j_create_statement_sub6 = "create (sub:SubRune {SubRuneName :"+ repr(i[9]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub6)
  neo4j_create_statement_sub7 = "create (sub:SubRune {SubRuneName :"+ repr(i[10]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub7)
  neo4j_create_statement_sub8 = "create (sub:SubRune {SubRuneName :"+ repr(i[11]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub8)
  neo4j_create_statement_sub9 = "create (sub:SubRune {SubRuneName :"+ repr(i[12]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub9)
for i in neo_data_inspiration:
  neo4j_create_statement_tree1 = "create (tree:RuneTree {RuneTreeName :"+ repr(i[0]) +", RuneKeystone1 :"+ repr(i[1]) +", RuneKeystone2 :"+ repr(i[2]) +", RuneKeystone3 :"+ repr(i[3]) +", SubRune1 :"+ repr(i[4]) +", SubRune2 :"+ repr(i[5]) +", SubRune3 :"+ repr(i[6]) +", SubRune4 :"+ repr(i[7]) +", SubRune5 :"+ repr(i[8]) +", SubRune6 :"+ repr(i[9]) +", SubRune7 :"+ repr(i[10]) +", SubRune8 :"+ repr(i[11]) +", SubRune9 :"+ repr(i[12]) +"})"
  neo_exec_comm.append(neo4j_create_statement_tree1)
  neo4j_create_statement_key1 = "create (key:KeyStone {RuneKeystone :"+ repr(i[1]) +"})"
  neo_exec_comm.append(neo4j_create_statement_key1)
  neo4j_create_statement_key2 = "create (key:KeyStone {RuneKeystone :"+ repr(i[2]) +"})"
  neo_exec_comm.append(neo4j_create_statement_key2)
  neo4j_create_statement_key3 = "create (key:KeyStone {RuneKeystone :"+ repr(i[3]) +"})"
  neo_exec_comm.append(neo4j_create_statement_key3)
  neo4j_create_statement_sub1 = "create (sub:SubRune {SubRuneName :"+ repr(i[4]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub1)
  neo4j_create_statement_sub2 = "create (sub:SubRune {SubRuneName :"+ repr(i[5]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub2)
  neo4j_create_statement_sub3 = "create (sub:SubRune {SubRuneName :"+ repr(i[6]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub3)
  neo4j_create_statement_sub4 = "create (sub:SubRune {SubRuneName :"+ repr(i[7]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub4)
  neo4j_create_statement_sub5 = "create (sub:SubRune {SubRuneName :"+ repr(i[8]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub5)
  neo4j_create_statement_sub6 = "create (sub:SubRune {SubRuneName :"+ repr(i[9]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub6)
  neo4j_create_statement_sub7 = "create (sub:SubRune {SubRuneName :"+ repr(i[10]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub7)
  neo4j_create_statement_sub8 = "create (sub:SubRune {SubRuneName :"+ repr(i[11]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub8)
  neo4j_create_statement_sub9 = "create (sub:SubRune {SubRuneName :"+ repr(i[12]) +"})"
  neo_exec_comm.append(neo4j_create_statement_sub9)

def exec_neo_data(neo_exec_comm):
  data_base_connect = GraphDatabase.driver(uri = "bolt://localhost:7687", auth = ("neo4j", "test1"))#target on 'project' db, pls change destination
  session = data_base_connect.session()
  for i in neo_exec_comm:
    session.run(i)

exec_neo_data(neo_exec_comm)


In [9]:
link_rows

['https://u.gg/lol/champions/aatrox/build',
 'https://u.gg/lol/champions/ahri/build',
 'https://u.gg/lol/champions/akali/build',
 'https://u.gg/lol/champions/akshan/build',
 'https://u.gg/lol/champions/alistar/build',
 'https://u.gg/lol/champions/amumu/build',
 'https://u.gg/lol/champions/anivia/build',
 'https://u.gg/lol/champions/annie/build',
 'https://u.gg/lol/champions/aphelios/build',
 'https://u.gg/lol/champions/ashe/build',
 'https://u.gg/lol/champions/aurelionsol/build',
 'https://u.gg/lol/champions/aurora/build',
 'https://u.gg/lol/champions/azir/build',
 'https://u.gg/lol/champions/bard/build',
 'https://u.gg/lol/champions/belveth/build',
 'https://u.gg/lol/champions/blitzcrank/build',
 'https://u.gg/lol/champions/brand/build',
 'https://u.gg/lol/champions/braum/build',
 'https://u.gg/lol/champions/briar/build',
 'https://u.gg/lol/champions/caitlyn/build',
 'https://u.gg/lol/champions/camille/build',
 'https://u.gg/lol/champions/cassiopeia/build',
 'https://u.gg/lol/champion

In [10]:
TableForSpell

,0
0,Heal
1,Ghost
2,Barrier
3,Exhaust
4,Flash
5,Teleport
6,Smite
7,Cleanse
8,Ignite


In [94]:
neo_data_ShardRune = TableForShardRune.values.tolist()
for i in neo_data_ShardRune:
    print(i)


['Adaptive Force Shard']
['Attack Speed Shard']
['Scaling CDR Shard']
['Adaptive Force Shard']
['Armor Shard']
['Magic Resist Shard']
['Scaling Bonus Health Shard']
['Armor Shard']
['Magic Resist Shard']


In [11]:
use_primary_rune_tree_list = []
use_primary_rune_keystone_list = []
use_primary_rune_rune_list = []
use_secondary_rune_tree_list = []
use_secondary_rune_rune_list = []
use_shard_rune_rune_list = []
use_suggest_spell_text_list = []

for i in primary_rune_tree_list:
    x = i.replace('The Rune Tree ', '')
    use_primary_rune_tree_list.append(x)

for i in primary_rune_keystone_list:
    x = i.replace('The Keystone ', '')
    use_primary_rune_keystone_list.append(x)
    
for i in primary_rune_rune_list:
    for e in i:
        x = e.replace('The Rune ', '')
        use_primary_rune_rune_list.append(x)

for i in secondary_rune_tree_list:
    x = i.replace('The Rune Tree ', '')
    use_secondary_rune_tree_list.append(x)

for i in secondary_rune_rune_list:
    for e in i:
        x = e.replace('The Rune ', '')
        use_secondary_rune_rune_list.append(x)

for i in shard_rune_rune_list:
    for e in i:
        x = e.replace('The ', '')
        use_shard_rune_rune_list.append(x)

for i in suggest_spell_text_list:
    for e in i:
        x = e.replace('Summoner Spell ', '')
        use_suggest_spell_text_list.append(x)


In [12]:
use_suggest_spell_text_list

['Flash',
 'Teleport',
 'Flash',
 'Teleport',
 'Flash',
 'Teleport',
 'Flash',
 'Ignite',
 'Flash',
 'Ignite',
 'Flash',
 'Smite',
 'Flash',
 'Ignite',
 'Flash',
 'Ignite',
 'Flash',
 'Barrier',
 'Flash',
 'Barrier',
 'Flash',
 'Teleport',
 'Flash',
 'Teleport',
 'Flash',
 'Teleport',
 'Flash',
 'Ignite',
 'Flash',
 'Smite',
 'Flash',
 'Ignite',
 'Flash',
 'Ignite',
 'Flash',
 'Heal',
 'Flash',
 'Smite',
 'Flash',
 'Barrier',
 'Teleport',
 'Ignite',
 'Flash',
 'Teleport',
 'Flash',
 'Teleport',
 'Flash',
 'Teleport',
 'Flash',
 'Ghost',
 'Flash',
 'Smite',
 'Flash',
 'Teleport',
 'Flash',
 'Barrier',
 'Flash',
 'Smite',
 'Flash',
 'Smite',
 'Flash',
 'Smite',
 'Flash',
 'Barrier',
 'Flash',
 'Smite',
 'Flash',
 'Ignite',
 'Flash',
 'Ignite',
 'Flash',
 'Teleport',
 'Flash',
 'Teleport',
 'Flash',
 'Ignite',
 'Flash',
 'Teleport',
 'Flash',
 'Smite',
 'Flash',
 'Smite',
 'Teleport',
 'Ignite',
 'Ghost',
 'Smite',
 'Flash',
 'Teleport',
 'Flash',
 'Teleport',
 'Flash',
 'Teleport',
 'Fla